In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from isaacgym import gymapi
from nerf_grasping.sim import ig_objects, ig_utils
from nerf_grasping.sim.ig_robot import FingertipRobot
from nerf_grasping.sim import ig_viz_utils
from nerf_grasping import grasp_utils
import torch

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.11.0+cu113
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym-3/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/py37_cu113/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


In [3]:
gym = gymapi.acquire_gym()
sim = ig_utils.setup_sim(gym)
env = ig_utils.setup_env(gym, sim)
ig_utils.setup_stage(gym, sim, env)
viewer = ig_utils.setup_viewer(gym, sim, env)

Obj = ig_objects.Banana
grasp_points, grasp_normals = Obj.grasp_points, Obj.grasp_normals

def run_gym(t=450):
    for _ in range(t): ig_utils.step_gym(gym, sim, viewer)

# Creates the robot, fop objective, and object
# robot = FingertipRobot(gym, sim, env, grasp_vars=None, 
#                        norm_start_offset=0.)
obj = Obj(gym, sim, env)

# robot.setup_tensors()
obj.setup_tensors()
obj.load_nerf_model()
obj.load_trimesh()
run_gym(4)

[INFO] Trainer: ngp | 2022-06-10_17-54-26 | cuda:0 | fp32 | 
/scr-ssd/ksrini/nerf_grasping/torch-ngp/logs/banana

[INFO] #parameters: 12667002

[INFO] Loading latest checkpoint ...

[INFO] Latest checkpoint is 
/scr-ssd/ksrini/nerf_grasping/torch-ngp/logs/banana/checkpoints/ngp_ep0200.pth.tar

[INFO] loaded model.

[WARN] unexpected keys: ['density_grid', 'step_counter']

[WARN] Failed to load optimizer, use default.

[WARN] Failed to load scheduler, use default.

[INFO] loaded scaler.

In [4]:
grasps = np.load("../grasp_data/banana.npy")
grasp_idx = 0

grasp_points = torch.tensor(grasps[grasp_idx, :, :6], dtype=torch.float32)

In [5]:
nerf_grasp_points = grasp_utils.ig_to_nerf(grasp_points.reshape(6, 3)).reshape(3, 6)

# get grasp distribution from grasp_points
_, _, weights, z_vals = grasp_utils.get_grasp_distribution(
    nerf_grasp_points.reshape(1, 3, 6).cuda(),
    obj.model,
    residual_dirs=False,
)
pre_correction, post_correction = [], []


In [32]:
grasp_points[:, 3]

tensor([ 0.7470, -0.7930, -0.0613])

In [45]:
rays_o = grasp_points[:, :3].cuda()
rays_d = grasp_points[:, 3:].cuda()

des_z_dist = 0.06
# Correct rays_o to be equidistant from object surface using estimated z_vals from nerf
# z_correction shape: (number of fingers, 1)
z_correction = des_z_dist - torch.sum(weights * z_vals, dim=-1).reshape(3, 1)
rays_o += z_correction * rays_d

In [61]:
np.linalg.norm(ig_utils.get_mesh_contacts(obj.gt_mesh, grasp_points[:, :3])[0] - grasp_points[:, :3].numpy(), axis=1)[:, None]

array([[0.05218911],
       [0.02802941],
       [0.03098524]])

In [50]:
torch.sum(weights * z_vals, dim=-1).reshape(3, 1)

tensor([[0.0660],
        [0.0749],
        [0.0575]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)

In [46]:
pre_correction = ig_viz_utils.visualize_markers(gym, env, sim, positions=grasp_points[:, :3],
                               colors=[[1.,0.,0.]]*3, marker_handles=pre_correction)
post_correction = ig_viz_utils.visualize_markers(gym, env, sim, positions=rays_o,
                               colors=[[0.,1.,0.]]*3, marker_handles=post_correction)
run_gym(1)

In [48]:
run_gym(1000)